In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/sentipol_embedd_7410_128_768_alberto_training.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/dictionary_sentipolc_8266_training.joblib")

In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["oneg"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[1 1 1 ... 1 0 0]
[1 1 1 ... 1 0 0]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[1]


In [ ]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

In [ ]:
import joblib
import pandas as pd
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/dictionary_agritrend_8266.joblib")

In [ ]:
# define example
data = dataset_test["label_neg"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 0
 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0
 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 1 0
 0 0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1
 1 0 0 0 1 0 0 0 0 0 0 0 

In [ ]:
import keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(8266,))

#Bi-LSTM
#bi = Bidirectional(LSTM(64, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True) (bi)

app = Add() ([bi, att])                      
flat = Flatten()(app)
den1 = Dense(4000) (flat)
den2 = Dense(2000) (den1)
den3 = Dense(1000) (den2)
den4 = Dense(500) (den3)
den5 = Dense(256) (den4)
den6 = Dense(128) (den5)
den7 = Dense(64) (den6)

#WMAL 
d0 = Dense(1000) (input_wmal)

#Concat
concat = Concatenate(axis=1)([den7, d0])
final_dense = Dense(32, activation="relu") (concat)
output = Dense(2,activation="softmax") (final_dense)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.RMSprop()

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

class myCallback(keras.callbacks.Callback):
    bestF1=0
    def __init__(self):
        super().__init__()
        self._get_pred = None
        self.preds = []
    def on_epoch_end(self,epoch, logs=None):
        predictions = m.predict([emb_matrix_test,wmal_matrix_test])
        predictions_vec = np.zeros((986,4))
        i = 0
        confidences= []
        for item in predictions:
          index = np.argmax(item);
          confidences.append(item[index])
          predictions_vec[i][index] = 1
          i = i+1
        labels_to_eval = []
        for vec in predictions_vec:
          inverted = label_encoder.inverse_transform([argmax(vec)])[0]
          labels_to_eval.append(inverted)
        
        precision,recall,fscore,support=score(dataset_test["label_neg"],labels_to_eval,average='macro')
        print("Test f1-score:"+str(fscore))
        if fscore > self.bestF1:
          m.save("bestModel.h5")
          print("BEST MODEL!!!!!")
          self.bestF1 = fscore

callbacks = [myCallback()]

In [ ]:
m.summary()

In [ ]:
m.fit([emb_matrix_train,wmal_matrix_train],onehot_encoded,batch_size=128, epochs=100,validation_split=0.15, callbacks=[callbacks])
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_neg_sentipolc_0.72803.h5")

Epoch 1/100
50/50 [==============================] - 7s 92ms/step - loss: 71.2069 - accuracy: 0.5113 - val_loss: 0.4863 - val_accuracy: 0.8085
Test f1-score:0.4340693219983207
BEST MODEL!!!!!
Epoch 2/100
50/50 [==============================] - 4s 81ms/step - loss: 0.6351 - accuracy: 0.6846 - val_loss: 0.5778 - val_accuracy: 0.7338
Test f1-score:0.5180750302198456
BEST MODEL!!!!!
Epoch 3/100
50/50 [==============================] - 4s 73ms/step - loss: 1.4287 - accuracy: 0.6486 - val_loss: 0.5635 - val_accuracy: 0.8058
Test f1-score:0.4741431167172083
Epoch 4/100
50/50 [==============================] - 4s 73ms/step - loss: 0.9270 - accuracy: 0.6565 - val_loss: 0.6050 - val_accuracy: 0.7941
Test f1-score:0.4873620220764678
Epoch 5/100
50/50 [==============================] - 4s 72ms/step - loss: 0.5274 - accuracy: 0.7370 - val_loss: 0.5873 - val_accuracy: 0.8210
Test f1-score:0.5604057608015027
BEST MODEL!!!!!
Epoch 6/100
50/50 [==============================] - 4s 75ms/step - loss: 0.

In [ ]:
m.load_weights("bestModel.h5")
predictions = m.predict([emb_matrix_test,wmal_matrix_test])

In [ ]:
import numpy as np
predictions_vec = np.zeros((986,4))
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [ ]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["label_neg"],labels_to_eval,digits=5))

              precision    recall  f1-score   support

           0    0.81039   0.81268   0.81153       710
           1    0.51460   0.51087   0.51273       276

    accuracy                        0.72819       986
   macro avg    0.66250   0.66177   0.66213       986
weighted avg    0.72759   0.72819   0.72789       986



In [ ]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_new_neg_agritrend_0.67343.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_AGRITREND_neg_full_0.67343.csv");